In [203]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.preprocessing import FunctionTransformer
import joblib
from sklearn.metrics import classification_report

In [204]:
def get_script_from_id(id):
    script = open('../data/script/' + id + '.script', 'r').read()
    # print(script)
    script = script.replace("'", " ").replace('"', ' ').replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\b', ' ').replace('\\', ' ')
    return script

def get_pd_dataframe():
    inputFile = open('../data_gathering/baseline/output/imdb_id_with_age_rating_and_labels.txt')
    df_data = []
    for line in inputFile:
        line_data = line.strip().split(',')
        # print(line_data)
        line_data.append(int(line_data[3]) + int(line_data[4]) + int(line_data[5]) + int(line_data[6]))
        
        max_index = 0
        max_value = 0
        for i in range(3,7):
            vote_count = int(line_data[i])
            if(vote_count >= max_value):
                max_index = i - 3
                max_value = vote_count
        line_data.append(max_index)
        try:
            script = get_script_from_id(line_data[0])
        except:
            # print('Error on loading script for id: ' + line_data[0])
            continue
        line_data.append(script)
        df_data.append(line_data)

    # id | Aspect | None | Mild | Moderate | Severe | Total_votes | Aspect_rating | text
    df = pd.DataFrame(df_data, columns=['imdb_id', 'age_rating', 'aspect', 'votes none', 'votes mild', 'votes moderate', 'votes severe', 'total_votes', 'aspect_rating', 'text'])
    df.drop(columns=["age_rating"], inplace=True)
    df = df.astype({'votes mild':'int', 'votes moderate':'int', 'votes severe':'int', 'votes none':'int', 'total_votes':'int', 'aspect_rating':'int'})
    return df

In [206]:
class SVMMovieClassifier():

    def __init__(self, df, model_name, num_features):

        self.df = df
        # Define the column transformer
        self.num_features = num_features
        self.num_transformer = Pipeline(steps=[
            ('normalize', FunctionTransformer(lambda x: x[num_features].div(x['total_votes'], axis=0), validate=False)),
            ('scale', StandardScaler())
        ])

        self.text_transformer = Pipeline(steps=[
            ('preprocess', TextPreprocessor()),
            ('vectorize', TfidfVectorizer())
        ])

        self.preprocessor = ColumnTransformer(transformers=[
            # ('num', self.num_transformer, num_features),
            ('text', self.text_transformer, 'text')
        ])

        # Define the pipeline
        self.pipe = Pipeline(steps=[
            ('preprocess', self.preprocessor),
            ('clf', SVC())
        ])

        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None

    def split_data(self):
        # Split the data into training and testing sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.df.drop('aspect_rating', axis=1), self.df['aspect_rating'], test_size=0.2, random_state=42)

    def train(self):
        # Train the pipeline
        self.pipe.fit(self.X_train, self.y_train)

    def test(self, X_test = None, y_test = None):
        # Test the pipeline
        if X_test is None or y_test is None:
            X_test = self.X_test
            y_test = self.y_test

        y_pred = self.pipe.predict(X_test)
        print(classification_report(y_test, y_pred))

    def save_model(self, model_name):
        # Save the pipeline
        joblib.dump(self.pipe, model_name)

    def load_model(self, model_name):
        # Load the pipeline
        self.pipe = joblib.load(model_name)

In [207]:
# Load the data
df = get_pd_dataframe()

# Drop rows with less than 5 votes
rows_to_drop = df.loc[df['total_votes'] < 5].index
df = df.drop(rows_to_drop)

# group the dataframe by 'aspect' and create a dictionary of dataframes
df_dict = {aspect: aspect_df.drop('aspect', axis=1) for aspect, aspect_df in df.groupby('aspect')}

aspect_classifier_dict = {}

# print the dictionary of dataframes
for aspect, aspect_df in df_dict.items():
    print(aspect, len(aspect_df))
    # Limit the number of rows to 50 for testing
    aspect_classifier_dict[aspect] = SVMMovieClassifier(aspect_df, 'movie_svm_' + aspect, ['votes none', 'votes mild', 'votes moderate', 'votes severe', 'total_votes'])


alcohol 477
frightening 475
nudity 478
profanity 477
violence 477


In [208]:
# Train the model
for aspect, movie_classifier in aspect_classifier_dict.items():
    try:
        print('Training model for aspect: ' + aspect)
        movie_classifier.split_data()
        movie_classifier.train()
    except Exception as e:
        print('Error on training model for aspect: ' + aspect)
        print(e)

Training model for aspect: alcohol
Training model for aspect: frightening
Training model for aspect: nudity
Training model for aspect: profanity
Training model for aspect: violence


In [209]:
# Test the model
for aspect, movie_classifier in aspect_classifier_dict.items():
    try:
        print('Testing model for aspect: ' + aspect)
        movie_classifier.test()
    except Exception as e:
        print('Error on testing model for aspect: ' + aspect + ' Error: ' + str(e))
        print(e)

Testing model for aspect: alcohol


/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.57      1.00      0.73        55
           2       0.00      0.00      0.00        23
           3       0.00      0.00      0.00         5

    accuracy                           0.57        96
   macro avg       0.14      0.25      0.18        96
weighted avg       0.33      0.57      0.42        96

Testing model for aspect: frightening


/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.00      0.00      0.00        26
           2       0.33      0.97      0.49        32
           3       0.00      0.00      0.00        24

    accuracy                           0.33        95
   macro avg       0.08      0.24      0.12        95
weighted avg       0.11      0.33      0.17        95

Testing model for aspect: nudity


/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.50      0.04      0.07        28
           1       0.40      0.97      0.57        39
           2       0.00      0.00      0.00        18
           3       0.00      0.00      0.00        11

    accuracy                           0.41        96
   macro avg       0.23      0.25      0.16        96
weighted avg       0.31      0.41      0.25        96

Testing model for aspect: profanity


/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       1.00      0.12      0.22         8
           1       0.39      0.91      0.55        33
           2       0.32      0.21      0.26        28
           3       0.00      0.00      0.00        27

    accuracy                           0.39        96
   macro avg       0.43      0.31      0.26        96
weighted avg       0.31      0.39      0.28        96

Testing model for aspect: violence
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.60      0.11      0.18        28
           2       0.40      0.97      0.57        37
           3       1.00      0.05      0.09        22

    accuracy                           0.42        96
   macro avg       0.50      0.28      0.21        96
weighted avg       0.56      0.42      0.29        96



/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/leonremke/opt/anaconda3/envs/DSTA-Ex/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo